In [1]:
import numpy as np
import autograd as ag
import autograd.numpy as anp
import scipy.optimize as opt
from sklearn.metrics.pairwise import rbf_kernel

In [2]:
# Please make sure that the package autograd is installed. If not, install it via the command !pip install autograd

In [3]:
# Instance 1

class MTL_H_Q:
    """ 
    Multi-Task Learning by Distribution Alignment (hinge loss, quadratic seed function)
    Parameters
    ----------
    sigma: float, default: 1.0
        Kernel bandwidth of the rbf kernel.
    lamda: float, default: 1.0
        Tradeoff parameter.
    gamma: float, default: 1e-3
        Regularization parameter.    
        
    """ 
    def __init__(self,sigma=1.0,lamda=1.0,gamma=1e-3):
        args_values = locals()
        args_values.pop("self")
        for arg,value in args_values.items():
            setattr(self,arg,value) 
    
    def fit(self, Xt, yt, X, y):
        """Fit the models according to the data.
        
        Parameters
        ----------
        Xt : array, shape = [mt, d]
             Data from task t, where mt is the number of samples
             and d is the number of features.

        yt : array-like, shape = [mt]
             Class labels from task t.
        
        X : array, shape = [m, d]
            Data from all tasks, where m is the number of samples
            and d is the number of features.

        y : array-like, shape = [m]
            Class labels from all tasks.
        
        Returns
        -------
        self : returns an instance of self.
        """
        
        m = X.shape[0]  
        
        # construct the feature kernel matrix   
        K,Kt = rbf_kernel(X,X,1./self.sigma),rbf_kernel(Xt,X,1./self.sigma)   
            
        # construct the label kernel matrix
        D,Dt = 1.0 * (y[:,None]==y), 1.0 * (yt[:,None]==y)
            
        def obj(vec): 
            hinge_loss = anp.mean(anp.exp(anp.dot(K * D,vec[m:])) * anp.maximum(0,1.0 - y * anp.dot(K,vec[:m])))
            quadratic_loss = 0.5 * anp.mean(anp.exp(2 * anp.dot(K * D,vec[m:]))) - anp.mean(anp.exp(anp.dot(Kt * Dt,vec[m:])))
            regularizer = anp.dot(vec[:m],vec[:m]) + anp.dot(vec[m:],vec[m:])                    
            return hinge_loss + self.lamda * quadratic_loss + self.gamma * regularizer     
            
        res = opt.minimize(fun=obj,jac=ag.grad(obj), x0=1e-3 * np.random.rand(2 * m) ,method='L-BFGS-B')
        
        self.alpha = res.x[:m]
        self.X = X
        
        return self
        
    def predict(self, Xt):
        """ 
        Parameters
        ----------
        Xt : array, shape = [mt, d]
             Data from task t, where mt is the number of samples
             and d is the number of features.

        Returns
        -------
        y_pred : array, shape = [mt]
                 Predicted class labels for data in Xt.
        """
        
        # construct the feature kernel matrix       
        Kt = rbf_kernel(Xt,self.X,1./self.sigma)   

        predictions = np.where(Kt.dot(self.alpha) < 0,-1,1)
        return predictions
    
    def score(self,Xt,yt):
        """
        Parameters
        ----------
        Xt : array, shape = [mt, d]
        
        yt : array-like, shape = [mt]
        Returns
        -------
        test score : scalar
        """ 
        y_hat = self.predict(Xt)
        acc = np.mean(yt==y_hat)
        return acc  

In [4]:
# Instance 2

class MTL_E_L:
    """ 
    Multi-Task Learning by Distribution Alignment (exponential loss, log seed function)
    Parameters
    ----------
    sigma: float, default: 1.0
        Kernel bandwidth of the rbf kernel.
    lamda: float, default: 1.0
        Tradeoff parameter.
    gamma: float, default: 1e-3
        Regularization parameter.    
        
    """ 
    def __init__(self,sigma=1.0,lamda=1.0,gamma=1e-3):
        args_values = locals()
        args_values.pop("self")
        for arg,value in args_values.items():
            setattr(self,arg,value) 
    
    def fit(self, Xt, yt, X, y):
        """Fit the models according to the data.
        
        Parameters
        ----------
        Xt : array, shape = [mt, d]
             Data from task t, where mt is the number of samples
             and d is the number of features.

        yt : array-like, shape = [mt]
             Class labels from task t.
        
        X : array, shape = [m, d]
            Data from all tasks, where m is the number of samples
            and d is the number of features.

        y : array-like, shape = [m]
            Class labels from all tasks.
        
        Returns
        -------
        self : returns an instance of self.
        """
        
        m = X.shape[0]  
        
        # construct the feature kernel matrix   
        K,Kt = rbf_kernel(X,X,1./self.sigma),rbf_kernel(Xt,X,1./self.sigma)   
            
        # construct the label kernel matrix
        D,Dt = 1.0 * (y[:,None]==y), 1.0 * (yt[:,None]==y)
            
        def obj(vec): 
            hinge_loss = anp.mean(anp.exp(anp.dot(K * D,vec[m:])) * anp.exp(- y * anp.dot(K,vec[:m])))
            quadratic_loss = anp.mean(anp.exp(anp.dot(K * D,vec[m:]))) - anp.mean(anp.dot(Kt * Dt,vec[m:]))
            regularizer = anp.dot(vec[:m],vec[:m]) + anp.dot(vec[m:],vec[m:])                    
            return hinge_loss + self.lamda * quadratic_loss + self.gamma * regularizer     
            
        res = opt.minimize(fun=obj,jac=ag.grad(obj), x0=1e-3 * np.random.rand(2 * m) ,method='L-BFGS-B')
        
        self.alpha = res.x[:m]
        self.X = X
        
        return self
        
    def predict(self, Xt):
        """ 
        Parameters
        ----------
        Xt : array, shape = [mt, d]
             Data from task t, where mt is the number of samples
             and d is the number of features.

        Returns
        -------
        y_pred : array, shape = [mt]
                 Predicted class labels for data in Xt.
        """
        
        # construct the feature kernel matrix       
        Kt = rbf_kernel(Xt,self.X,1./self.sigma)   

        predictions = np.where(Kt.dot(self.alpha) < 0,-1,1)
        return predictions
    
    def score(self,Xt,yt):
        """
        Parameters
        ----------
        Xt : array, shape = [mt, d]
        
        yt : array-like, shape = [mt]
        Returns
        -------
        test score : scalar
        """ 
        y_hat = self.predict(Xt)
        acc = np.mean(yt==y_hat)
        return acc  

In [5]:
import numpy as np
import pandas as pd
import scipy.io as sio
import numpy.linalg as la
from sklearn.svm import LinearSVC,SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [6]:
def readData(task,alltasks):
    scaler = StandardScaler()
    prefix = 'C:/Users/sentao/NoteBook3/8th work/'
    
    data = sio.loadmat(prefix + 'datasets/PIE/' + task + '.mat')
    Xt,yt = data['fea'].astype(np.float64),data['gnd'].ravel().astype(np.float64)
    Xt,yt = np.vstack((Xt[yt<=30],Xt[yt>30])),np.hstack((yt[yt<=30],yt[yt>30]))
    yt[yt<=30],yt[yt>30] = -1,1
    Xt, Xtest, yt, ytest = train_test_split(Xt, yt, test_size=0.8,random_state=0)
    
    X_list,y_list = [Xt],[yt]
    for t in alltasks:
        if t!=task:
            data = sio.loadmat(prefix + 'datasets/PIE/' + t + '.mat')
            Xt,yt = data['fea'].astype(np.float64),data['gnd'].ravel().astype(np.float64)
            Xt,yt = np.vstack((Xt[yt<=30],Xt[yt>30])),np.hstack((yt[yt<=30],yt[yt>30]))
            yt[yt<=30],yt[yt>30] = -1,1 
            X_list.append(Xt),y_list.append(yt)
            
    X, y = np.concatenate(X_list),np.concatenate(y_list)      
    scaler.fit(X)
    Xt, Xtest, X = scaler.transform(Xt),scaler.transform(Xtest),scaler.transform(X)
    
    return Xt,yt,X,y,Xtest,ytest

alltasks = ['PIE05','PIE27','PIE29']

In [7]:
for task in alltasks:
    Xt,yt,X,y,Xtest,ytest = readData(task,alltasks)      
    clf = MTL_H_Q(sigma=0.5,lamda=1,gamma=1e-2)
    print(task,'svm:', SVC(kernel='linear').fit(X,y).score(Xtest,ytest),'ours:',clf.fit(Xt,yt,X,y).score(Xtest,ytest))

PIE05 svm: 0.8484621155288822 ours: 0.8432108027006752
PIE27 svm: 0.8667417417417418 ours: 0.8412162162162162
PIE29 svm: 0.8093415007656968 ours: 0.7825421133231241


In [9]:
for task in alltasks:
    Xt,yt,X,y,Xtest,ytest = readData(task,alltasks)      
    clf = MTL_H_Q(sigma=5,lamda=1,gamma=1e-2)
    print(task,'svm:', SVC(kernel='linear').fit(X,y).score(Xtest,ytest),'ours:',clf.fit(Xt,yt,X,y).score(Xtest,ytest))

PIE05 svm: 0.8484621155288822 ours: 0.8570892723180795
PIE27 svm: 0.8667417417417418 ours: 0.850975975975976
PIE29 svm: 0.8093415007656968 ours: 0.781010719754977


In [ ]:
for task in alltasks:
    Xt,yt,X,y,Xtest,ytest = readData(task,alltasks)      
    clf = MTL_E_L(sigma=0.5,lamda=1,gamma=1e-2)
    print(task,'svm:', SVC(kernel='linear').fit(X,y).score(Xtest,ytest),'ours:',clf.fit(Xt,yt,X,y).score(Xtest,ytest))

In [ ]:
for task in alltasks:
    Xt,yt,X,y,Xtest,ytest = readData(task,alltasks)      
    clf = MTL_E_L(sigma=5,lamda=1,gamma=1e-2)
    print(task,'svm:', SVC(kernel='linear').fit(X,y).score(Xtest,ytest),'ours:',clf.fit(Xt,yt,X,y).score(Xtest,ytest))